In [2]:
pip install geopy

     |████████████████████████████████| 112kB 5.1MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install beautifulsoup4

     |████████████████████████████████| 112kB 8.5MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [6]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
import urllib.request
import json
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.colors as colors
%matplotlib inline
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes
import folium

print('Libraries imported.')

Libraries imported.


**For this part I will explore a dataset with a total of 5 boroughs and 306 neighborhoods. I will segement the neighborhoods and explore them with a dataset from https://geo.nyu.edu/catalog/nyu_2451_34572 that has the boroughs and the neighborhoods that exist in each borough as well as the the latitude and logitude coordinates of each neighborhood. The intent of this project is to analyze each neighborhood and the restaurants within those neighbrhoods. From this data one can see which types of restaurants are least common in certain neighborhoods. This data would be good for anyone looking to determine where they want to open a new restaurant based on type. In order to minimize competition and maximize customers, it would likely be best to open up a restaurant where there are least amount of the same type. For example, if you look at the data, you can see that there are very few Hawaiian restaurants in many of these neighborhoods and places as the 10th most common restaurant in many of these neighborhoods. This means anyone looking to open a Hawaiian restaurant would have multiple neighborhoods to choose from and have minimal competition. On the other end of the spectrum, sushi restaurants are very common in just about every neighborhood, making a sushi restaurant a much higher risk investment due to a high level of competition.**

In [7]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

Data downloaded!


In [8]:
neighborhoods_data = newyork_data['features']
column_names = ['Borough', 'Neighborhood', 'Laitude', 'Longitude']
neighborhoods = pd.DataFrame(columns=column_names)
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough']
    neighborhood_name = data['properties']['name']
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [9]:
neighborhoods.head()

,Borough,Neighborhood,Laitude,Longitude,Latitude
0,Bronx,Wakefield,NaN,-73.847201,40.894705
1,Bronx,Co-op City,NaN,-73.829939,40.874294
2,Bronx,Eastchester,NaN,-73.827806,40.887556
3,Bronx,Fieldston,NaN,-73.905643,40.895437
4,Bronx,Riverdale,NaN,-73.912585,40.890834


In [11]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [12]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Laitude,Longitude,Latitude
0,Manhattan,Marble Hill,NaN,-73.910660,40.876551
1,Manhattan,Chinatown,NaN,-73.994279,40.715618
2,Manhattan,Washington Heights,NaN,-73.936900,40.851903
3,Manhattan,Inwood,NaN,-73.921210,40.867684
4,Manhattan,Hamilton Heights,NaN,-73.949688,40.823604


In [13]:
import folium
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Borough'], manhattan_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork


In [15]:

import urllib
def getNearbyVenues(names, latitudes, longitudes, radius=5000, categoryIds=''):
    try:
        venues_list=[]
        for name, lat, lng in zip(names, latitudes, longitudes):
            #print(name)

            # create the API request URL
            url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)

            if (categoryIds != ''):
                url = url + '&categoryId={}'
                url = url.format(categoryIds)

            # make the GET request
            response = requests.get(url).json()
            results = response["response"]['venues']

            # return only relevant information for each nearby venue
            for v in results:
                success = False
                try:
                    category = v['categories'][0]['name']
                    success = True
                except:
                    pass

                if success:
                    venues_list.append([(
                        name, 
                        lat, 
                        lng, 
                        v['name'], 
                        v['location']['lat'], 
                        v['location']['lng'],
                        v['categories'][0]['name']
                    )])

        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude',  
                  'Venue Category']
    
    except:
        print(url)
        print(response)
        print(results)
        print(nearby_venues)

    return(nearby_venues)

In [18]:

LIMIT = 500 
radius = 5000 
CLIENT_ID = 'ZMHWBS0SR12Z3YDYVHJVTZPRK3U1ZP3I2TYQAJ5CU3JUHMB5'
CLIENT_SECRET = 'H3TT0XT3P5TIAFCV1Y2UUVLF42N44DICNKLUELK34H2TKLFR'
VERSION = '20181020'

In [19]:
#https://developer.foursquare.com/docs/resources/categories
#Sushi = 4bf58dd8d48988d1d2941735
neighborhoods = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
newyork_venues_sushi = getNearbyVenues(names=neighborhoods['Neighborhood'], latitudes=neighborhoods['Latitude'], longitudes=neighborhoods['Longitude'], radius=1000, categoryIds='4bf58dd8d48988d1d2941735')
newyork_venues_sushi.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.910660,Planet Tokyo,40.886233,-73.909479,Sushi Restaurant
1,Chinatown,40.715618,-73.994279,Shinsen,40.715608,-73.996611,Japanese Restaurant
2,Chinatown,40.715618,-73.994279,Shōji,40.717582,-74.005592,Sushi Restaurant
3,Chinatown,40.715618,-73.994279,Amano Sushi,40.716017,-73.992818,Sushi Restaurant
4,Chinatown,40.715618,-73.994279,Whole Foods Sushi Bar,40.724000,-73.992277,Sushi Restaurant


In [21]:
newyork_venues_sushi.shape

(1090, 7)

In [22]:
def addToMap(df, color, existingMap):
    for lat, lng, local, venue, venueCat in zip(df['Venue Latitude'], df['Venue Longitude'], df['Neighborhood'], df['Venue'], df['Venue Category']):
        label = '{} ({}) - {}'.format(venue, venueCat, local)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7).add_to(existingMap)

In [23]:

map_newyork_sushi = folium.Map(location=[latitude, longitude], zoom_start=10)
addToMap(newyork_venues_sushi, 'red', map_newyork_sushi)

map_newyork_sushi

In [24]:
def addColumn(startDf, columnTitle, dataDf):
    grouped = dataDf.groupby('Neighborhood').count()
    
    for n in startDf['Neighborhood']:
        try:
            startDf.loc[startDf['Neighborhood'] == n,columnTitle] = grouped.loc[n, 'Venue']
        except:
            startDf.loc[startDf['Neighborhood'] == n,columnTitle] = 0

In [25]:
manhattan_grouped = newyork_venues_sushi.groupby('Neighborhood').count()
manhattan_grouped
#print('There are {} uniques categories.'.format(len(newyork_venues_sushi['Venue Category'].unique())))

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,22,22,22,22,22,22
Carnegie Hill,21,21,21,21,21,21
Central Harlem,3,3,3,3,3,3
Chelsea,40,40,40,40,40,40
Chinatown,26,26,26,26,26,26
Civic Center,31,31,31,31,31,31
Clinton,38,38,38,38,38,38
East Harlem,3,3,3,3,3,3
East Village,50,50,50,50,50,50


In [26]:
# one hot encoding
manhattan_onehot = pd.get_dummies(newyork_venues_sushi[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = newyork_venues_sushi['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,Asian Restaurant,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Grocery Store,Hawaiian Restaurant,Japanese Restaurant,Noodle House,Poke Place,Ramen Restaurant,Restaurant,Sake Bar,Sandwich Place,Seafood Restaurant,Smoothie Shop,Steakhouse,Sushi Restaurant,Theme Restaurant,Vegetarian / Vegan Restaurant
0,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,Chinatown,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2,Chinatown,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,Chinatown,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,Chinatown,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [27]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

,Neighborhood,Asian Restaurant,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Grocery Store,Hawaiian Restaurant,Japanese Restaurant,Noodle House,Poke Place,Ramen Restaurant,Restaurant,Sake Bar,Sandwich Place,Seafood Restaurant,Smoothie Shop,Steakhouse,Sushi Restaurant,Theme Restaurant,Vegetarian / Vegan Restaurant
0,Battery Park City,0.000000,0.00,0.000000,0.000000,0.00000,0.000000,0.000000,0.090909,0.045455,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.818182,0.045455,0.000000
1,Carnegie Hill,0.047619,0.00,0.047619,0.000000,0.00000,0.000000,0.000000,0.095238,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.809524,0.000000,0.000000
2,Central Harlem,0.000000,0.00,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
3,Chelsea,0.050000,0.00,0.000000,0.000000,0.00000,0.000000,0.000000,0.100000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.825000,0.000000,0.025000
4,Chinatown,0.000000,0.00,0.000000,0.000000,0.00000,0.000000,0.000000,0.192308,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.807692,0.000000,0.000000
5,Civic Center,0.000000,0.00,0.000000,0.000000,0.00000,0.000000,0.000000,0.096774,0.032258,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.838710,0.032258,0.000000
6,Clinton,0.026316,0.00,0.026316,0.026316,0.00000,0.000000,0.000000,0.105263,0.000000,0.026316,0.00,0.00,0.000000,0.00,0.026316,0.000000,0.000000,0.763158,0.000000,0.000000
7,East Harlem,0.000000,0.00,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
8,East Village,0.000000,0.00,0.000000,0.000000,0.00000,0.000000,0.000000,0.120000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.860000,0.000000,0.020000
9,Financial District,0.000000,0.00,0.000000,0.000000,0.00000,0.000000,0.000000,0.181818,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.818182,0.000000,0.000000


In [28]:

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [29]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Sushi Restaurant,Japanese Restaurant,Noodle House,Theme Restaurant,Vegetarian / Vegan Restaurant,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Grocery Store
1,Carnegie Hill,Sushi Restaurant,Japanese Restaurant,Asian Restaurant,Chinese Restaurant,Noodle House,Bakery,Cocktail Bar,Deli / Bodega,Grocery Store,Hawaiian Restaurant
2,Central Harlem,Sushi Restaurant,Vegetarian / Vegan Restaurant,Noodle House,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Grocery Store,Hawaiian Restaurant,Japanese Restaurant
3,Chelsea,Sushi Restaurant,Japanese Restaurant,Asian Restaurant,Vegetarian / Vegan Restaurant,Smoothie Shop,Seafood Restaurant,Sandwich Place,Sake Bar,Restaurant,Ramen Restaurant
4,Chinatown,Sushi Restaurant,Japanese Restaurant,Vegetarian / Vegan Restaurant,Noodle House,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Grocery Store,Hawaiian Restaurant


In [30]:
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 2, 1, 1, 1, 0, 2, 1, 1], dtype=int32)

In [31]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = manhattan_data
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head()

,Borough,Neighborhood,Laitude,Longitude,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,NaN,-73.910660,40.876551,2,Sushi Restaurant,Vegetarian / Vegan Restaurant,Noodle House,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Grocery Store,Hawaiian Restaurant,Japanese Restaurant
1,Manhattan,Chinatown,NaN,-73.994279,40.715618,1,Sushi Restaurant,Japanese Restaurant,Vegetarian / Vegan Restaurant,Noodle House,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Grocery Store,Hawaiian Restaurant
2,Manhattan,Washington Heights,NaN,-73.936900,40.851903,2,Sushi Restaurant,Vegetarian / Vegan Restaurant,Noodle House,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Grocery Store,Hawaiian Restaurant,Japanese Restaurant
3,Manhattan,Inwood,NaN,-73.921210,40.867684,2,Sushi Restaurant,Vegetarian / Vegan Restaurant,Noodle House,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Grocery Store,Hawaiian Restaurant,Japanese Restaurant
4,Manhattan,Hamilton Heights,NaN,-73.949688,40.823604,2,Sushi Restaurant,Vegetarian / Vegan Restaurant,Noodle House,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Grocery Store,Hawaiian Restaurant,Japanese Restaurant


In [32]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [33]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Lenox Hill,0,Sushi Restaurant,Asian Restaurant,Japanese Restaurant,Noodle House,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Grocery Store,Hawaiian Restaurant
14,Clinton,0,Sushi Restaurant,Japanese Restaurant,Poke Place,Chinese Restaurant,Cocktail Bar,Asian Restaurant,Seafood Restaurant,Sandwich Place,Sake Bar,Restaurant
15,Midtown,0,Sushi Restaurant,Asian Restaurant,Seafood Restaurant,Japanese Restaurant,Ramen Restaurant,Bakery,Vegetarian / Vegan Restaurant,Sandwich Place,Sake Bar,Restaurant
16,Murray Hill,0,Sushi Restaurant,Japanese Restaurant,Asian Restaurant,Restaurant,Bakery,Chinese Restaurant,Ramen Restaurant,Vegetarian / Vegan Restaurant,Sake Bar,Sandwich Place
33,Midtown South,0,Sushi Restaurant,Asian Restaurant,Japanese Restaurant,Restaurant,Bakery,Chinese Restaurant,Ramen Restaurant,Vegetarian / Vegan Restaurant,Sake Bar,Sandwich Place
34,Sutton Place,0,Sushi Restaurant,Asian Restaurant,Japanese Restaurant,Steakhouse,Seafood Restaurant,Deli / Bodega,Bakery,Chinese Restaurant,Cocktail Bar,Grocery Store
39,Hudson Yards,0,Sushi Restaurant,Asian Restaurant,Japanese Restaurant,Cocktail Bar,Poke Place,Seafood Restaurant,Sandwich Place,Sake Bar,Restaurant,Ramen Restaurant


In [34]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Chinatown,1,Sushi Restaurant,Japanese Restaurant,Vegetarian / Vegan Restaurant,Noodle House,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Grocery Store,Hawaiian Restaurant
8,Upper East Side,1,Sushi Restaurant,Japanese Restaurant,Asian Restaurant,Noodle House,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Grocery Store,Hawaiian Restaurant
9,Yorkville,1,Sushi Restaurant,Japanese Restaurant,Asian Restaurant,Chinese Restaurant,Noodle House,Bakery,Cocktail Bar,Deli / Bodega,Grocery Store,Hawaiian Restaurant
13,Lincoln Square,1,Sushi Restaurant,Japanese Restaurant,Smoothie Shop,Chinese Restaurant,Grocery Store,Vegetarian / Vegan Restaurant,Noodle House,Bakery,Cocktail Bar,Deli / Bodega
17,Chelsea,1,Sushi Restaurant,Japanese Restaurant,Asian Restaurant,Vegetarian / Vegan Restaurant,Smoothie Shop,Seafood Restaurant,Sandwich Place,Sake Bar,Restaurant,Ramen Restaurant
19,East Village,1,Sushi Restaurant,Japanese Restaurant,Vegetarian / Vegan Restaurant,Noodle House,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Grocery Store,Hawaiian Restaurant
21,Tribeca,1,Sushi Restaurant,Noodle House,Japanese Restaurant,Theme Restaurant,Vegetarian / Vegan Restaurant,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Grocery Store
22,Little Italy,1,Sushi Restaurant,Japanese Restaurant,Noodle House,Vegetarian / Vegan Restaurant,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Grocery Store,Hawaiian Restaurant
23,Soho,1,Sushi Restaurant,Japanese Restaurant,Noodle House,Theme Restaurant,Vegetarian / Vegan Restaurant,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Grocery Store
24,West Village,1,Sushi Restaurant,Japanese Restaurant,Vegetarian / Vegan Restaurant,Sake Bar,Asian Restaurant,Seafood Restaurant,Sandwich Place,Smoothie Shop,Restaurant,Ramen Restaurant


In [35]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Marble Hill,2,Sushi Restaurant,Vegetarian / Vegan Restaurant,Noodle House,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Grocery Store,Hawaiian Restaurant,Japanese Restaurant
2,Washington Heights,2,Sushi Restaurant,Vegetarian / Vegan Restaurant,Noodle House,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Grocery Store,Hawaiian Restaurant,Japanese Restaurant
3,Inwood,2,Sushi Restaurant,Vegetarian / Vegan Restaurant,Noodle House,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Grocery Store,Hawaiian Restaurant,Japanese Restaurant
4,Hamilton Heights,2,Sushi Restaurant,Vegetarian / Vegan Restaurant,Noodle House,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Grocery Store,Hawaiian Restaurant,Japanese Restaurant
5,Manhattanville,2,Sushi Restaurant,Vegetarian / Vegan Restaurant,Noodle House,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Grocery Store,Hawaiian Restaurant,Japanese Restaurant
6,Central Harlem,2,Sushi Restaurant,Vegetarian / Vegan Restaurant,Noodle House,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Grocery Store,Hawaiian Restaurant,Japanese Restaurant
7,East Harlem,2,Sushi Restaurant,Vegetarian / Vegan Restaurant,Noodle House,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Grocery Store,Hawaiian Restaurant,Japanese Restaurant


In [36]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Upper West Side,3,Sushi Restaurant,Japanese Restaurant,Asian Restaurant,Grocery Store,Noodle House,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Hawaiian Restaurant
20,Lower East Side,3,Sushi Restaurant,Japanese Restaurant,Vegetarian / Vegan Restaurant,Noodle House,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Grocery Store,Hawaiian Restaurant


In [37]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Roosevelt Island,4,Sushi Restaurant,Asian Restaurant,Noodle House,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Grocery Store,Hawaiian Restaurant,Japanese Restaurant
18,Greenwich Village,4,Sushi Restaurant,Japanese Restaurant,Sake Bar,Vegetarian / Vegan Restaurant,Noodle House,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Grocery Store
26,Morningside Heights,4,Sushi Restaurant,Hawaiian Restaurant,Vegetarian / Vegan Restaurant,Noodle House,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Grocery Store,Japanese Restaurant
35,Turtle Bay,4,Sushi Restaurant,Japanese Restaurant,Asian Restaurant,Steakhouse,Seafood Restaurant,Bakery,Chinese Restaurant,Cocktail Bar,Deli / Bodega,Grocery Store
36,Tudor City,4,Sushi Restaurant,Asian Restaurant,Japanese Restaurant,Vegetarian / Vegan Restaurant,Steakhouse,Smoothie Shop,Seafood Restaurant,Sandwich Place,Sake Bar,Restaurant
